In [1]:
import feedparser
from bs4 import BeautifulSoup
import urllib
from dateparser import parse as parse_date
import requests



class GoogleNews:
    def __init__(self, lang = 'en', country = 'US'):
        self.lang = lang.lower()
        self.country = country.upper()
        self.BASE_URL = 'https://news.google.com/rss'

    def __top_news_parser(self, text):
        """Return subarticles from the main and topic feeds"""
        try:
            bs4_html = BeautifulSoup(text, "html.parser")
            # find all li tags
            lis = bs4_html.find_all('li')
            sub_articles = []
            for li in lis:
                try:
                    sub_articles.append({"url": li.a['href'],
                                         "title": li.a.text,
                                         "publisher": li.font.text})
                except:
                    pass
            return sub_articles
        except:
            return text

    def __ceid(self):
        """Compile correct country-lang parameters for Google News RSS URL"""
        return '?ceid={}:{}&hl={}&gl={}'.format(self.country,self.lang,self.lang,self.country)

    def __add_sub_articles(self, entries):
        for i, val in enumerate(entries):
            if 'summary' in entries[i].keys():
                entries[i]['sub_articles'] = self.__top_news_parser(entries[i]['summary'])
            else:
                entries[i]['sub_articles'] = None
        return entries

    def __scaping_bee_request(self, api_key, url):
        response = requests.get(
            url="https://app.scrapingbee.com/api/v1/",
            params={
                "api_key": api_key,
                "url": url,
                "render_js": "false"
            }
        )
        if response.status_code == 200:
            return response
        if response.status_code != 200:
            raise Exception("ScrapingBee status_code: "  + str(response.status_code) + " " + response.text)

    def __parse_feed(self, feed_url, proxies=None, scraping_bee = None):

        if scraping_bee and proxies:
            raise Exception("Pick either ScrapingBee or proxies. Not both!")

        if proxies:
            r = requests.get(feed_url, proxies = proxies)
        else:
            r = requests.get(feed_url)

        if scraping_bee:
            r = self.__scaping_bee_request(url = feed_url, api_key = scraping_bee)
        else:
            r = requests.get(feed_url)


        if 'https://news.google.com/rss/unsupported' in r.url:
            raise Exception('This feed is not available')

        d = feedparser.parse(r.text)

        if not scraping_bee and not proxies and len(d['entries']) == 0:
            d = feedparser.parse(feed_url)

        return dict((k, d[k]) for k in ('feed', 'entries'))

    def __search_helper(self, query):
        return urllib.parse.quote_plus(query)

    def __from_to_helper(self, validate=None):
        try:
            validate = parse_date(validate).strftime('%Y-%m-%d')
            return str(validate)
        except:
            raise Exception('Could not parse your date')



    def top_news(self, proxies=None, scraping_bee = None):
        """Return a list of all articles from the main page of Google News
        given a country and a language"""
        d = self.__parse_feed(self.BASE_URL + self.__ceid(), proxies=proxies, scraping_bee=scraping_bee)
        d['entries'] = self.__add_sub_articles(d['entries'])
        return d

    def topic_headlines(self, topic: str, proxies=None, scraping_bee=None):
        """Return a list of all articles from the topic page of Google News
        given a country and a language"""
        #topic = topic.upper()
        if topic.upper() in ['WORLD', 'NATION', 'BUSINESS', 'TECHNOLOGY', 'ENTERTAINMENT', 'SCIENCE', 'SPORTS', 'HEALTH']:
            d = self.__parse_feed(self.BASE_URL + '/headlines/section/topic/{}'.format(topic.upper()) + self.__ceid(), proxies = proxies, scraping_bee=scraping_bee)

        else:
            d = self.__parse_feed(self.BASE_URL + '/topics/{}'.format(topic) + self.__ceid(), proxies = proxies, scraping_bee=scraping_bee)

        d['entries'] = self.__add_sub_articles(d['entries'])
        if len(d['entries']) > 0:
            return d
        else:
            raise Exception('unsupported topic')

    def geo_headlines(self, geo: str, proxies=None, scraping_bee=None):
        """Return a list of all articles about a specific geolocation
        given a country and a language"""
        d = self.__parse_feed(self.BASE_URL + '/headlines/section/geo/{}'.format(geo) + self.__ceid(), proxies = proxies, scraping_bee=scraping_bee)

        d['entries'] = self.__add_sub_articles(d['entries'])
        return d

    def search(self, query: str, helper = True, when = None, from_ = None, to_ = None, proxies=None, scraping_bee=None):
        """
        Return a list of all articles given a full-text search parameter,
        a country and a language

        :param bool helper: When True helps with URL quoting
        :param str when: Sets a time range for the artiles that can be found
        """

        if when:
            query += ' when:' + when

        if from_ and not when:
            from_ = self.__from_to_helper(validate=from_)
            query += ' after:' + from_

        if to_ and not when:
            to_ = self.__from_to_helper(validate=to_)
            query += ' before:' + to_

        if helper == True:
            query = self.__search_helper(query)

        search_ceid = self.__ceid()
        search_ceid = search_ceid.replace('?', '&')

        d = self.__parse_feed(self.BASE_URL + '/search?q={}'.format(query) + search_ceid, proxies = proxies, scraping_bee=scraping_bee)

        d['entries'] = self.__add_sub_articles(d['entries'])
        return d

In [9]:
from pygooglenews import GoogleNews
import csv
import os

In [15]:
gn = GoogleNews(country='ID')

def get_titles(search):
    stories = []
    search = gn.search(search)
    newsitem = search['entries']
    for item in newsitem:
        story = {
            'title' : item.title,
            'link' :  item.link
        }
        stories.append(story)
    return stories

print(get_titles('Korupsi'))

[{'title': 'JK Mengaku Bingung dengan Kasus Dugaan Korupsi Karen Agustiawan - kompas.id', 'link': 'https://news.google.com/rss/articles/CBMirAFBVV95cUxQZW82QTd2eXMtaklCQ3ZXSnpLWXVKOFBmeGh3MUtoTHNpRnJ2dk5iZDZJN0JycE1SNHBNRDRYTTdsRWRHVjJHdHU0TmFwYkE3c3QtalZaUEp4SjlnNl9vdGxMQkdqUW9OZEhMa011Nk5ZemFrUl94cy1ZUmRFNk5mUHIwRk1ieVRKaUNTelhUVzVfTnZTOXBfc2NVak1qQVVVc0J1MjRWNjdIS256?oc=5'}, {'title': 'Sidang lanjutan kasus dugaan korupsi tata niaga komoditas timah - ANTARA FOTO', 'link': 'https://news.google.com/rss/articles/CBMipgFBVV95cUxOTHgzS0ptdTdRczcxSGJPNkJjQjB5TDJoZnFYN3B0ZU9DZC1oV1pQZ3hETVhURkZFWlpVN044MmFKV1Q2dmZjdGh1U280OFZEYjlYdGdvOWpIdlJJYTE1bE1RS1pzdm9qSjNZNTc5SEhiVnlrT1N0aXlnb2tKM0pGdnNzbEQ0Vm16M2thU2duTldXaVBJZzFPQm9mRTE1X2M3VFlfQlpR0gGrAUFVX3lxTE9wYUtGamlKZ1c4Z3cyMUlDQVFqd0hqeDFSem9oNDJTYzF6Uk9nSjRWOHFmZ1BpenBuMVNBSW45NHg2ZUl0OHdwdkhYeFVLRXl0SGRLSmVKaUNBX2pxRXd1VnBpSWhMRVdkLXZPalg2SHBPSGMteGp2YzRWMjhhcUw1eFJPOXZrRHlpNzVVZWc3ak1CQkRoNlR1WXJTWXV5LXhabmNxc1M5aE5waw?oc=5'}, {'title': '

In [16]:
def save_to_csv(stories, filename):
    # Create directory if it doesn't exist
    output_dir = "crawl-output"
    os.makedirs(output_dir, exist_ok=True)
    
    # Full path for the CSV file
    file_path = os.path.join(output_dir, filename)
    
    # Specify the CSV file header
    header = ['Title', 'Link']
    
    # Open the CSV file in write mode
    with open(file_path, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=header)
        writer.writeheader()  # Write header
        for story in stories:
            writer.writerow({'Title': story['title'], 'Link': story['link']})
    
    print(f"Data saved to {file_path}")

In [8]:
# Main Program
if __name__ == "__main__":
    search_query = 'Korupsi'  # Set your search term here
    stories = get_titles(search_query)
    
    # Save results to a CSV file in the 'crawler-output' directory
    csv_filename = f'{search_query}_news.csv'
    save_to_csv(stories, csv_filename)

Data saved to lockdown_news.csv
